In [10]:
import numpy as np
from tqdm import tqdm
import os
import tempfile
import pandas as pd

# Constants
NUM_NODES = 30_000_000
AVG_DEGREE = 10
CHUNK_SIZE = 1_000_000

def create_memory_mapped_edge_list(num_edges):
    # Create a memory-mapped array for edges
    fp = tempfile.NamedTemporaryFile(delete=False)
    fp.close()
    edge_array = np.memmap(fp.name, dtype=np.int32, mode='w+', shape=(num_edges, 2))
    return edge_array, fp.name

def generate_edges(edge_array, num_nodes, num_edges):
    print("Generating edges...")
    for i in tqdm(range(0, num_edges, CHUNK_SIZE)):
        chunk_size = min(CHUNK_SIZE, num_edges - i)
        chunk = np.random.randint(0, num_nodes, size=(chunk_size, 2))
        edge_array[i:i+chunk_size] = chunk
    print("Edge generation complete.")

def save_network_data_csv(filename='large_network.csv'):
    print(f"Saving network data to {filename}...")

    num_edges = NUM_NODES * AVG_DEGREE // 2

    # Generate and save edges
    edge_array, temp_file = create_memory_mapped_edge_list(num_edges)
    generate_edges(edge_array, NUM_NODES, num_edges)

    with open(filename, 'w') as f:
        f.write("source,target,weight\n")
        for i in tqdm(range(0, num_edges, CHUNK_SIZE)):
            chunk_size = min(CHUNK_SIZE, num_edges - i)
            edge_chunk = edge_array[i:i+chunk_size]
            weights = np.random.random(chunk_size).astype(np.float32)
            for j in range(chunk_size):
                f.write(f"{edge_chunk[j, 0]},{edge_chunk[j, 1]},{weights[j]}\n")

    # Clean up
    del edge_array
    os.unlink(temp_file)

    print("Data saving complete.")

# Main function to generate and save the network
def generate_and_save_large_network():
    save_network_data_csv()

# Run the generator
if __name__ == "__main__":
    generate_and_save_large_network()

Saving network data to large_network.csv...
Generating edges...


100%|██████████| 150/150 [00:01<00:00, 83.63it/s]


Edge generation complete.


100%|██████████| 150/150 [03:06<00:00,  1.24s/it]

Data saving complete.
